In [9]:
import findspark
findspark.init()

from pyspark.sql import SparkSession

# Initialize SparkSession
spark = SparkSession.builder \
    .appName("Recommendation System") \
    .getOrCreate()

# Load the CSV file
df = spark.read.csv("D:\Crack\Book_rating.csv", header=True, inferSchema=True)

# Define the list of columns you want to keep
columns_to_keep = ["id", "User_id", "review_score"]

# Select only the columns you want to keep
df_cleaned = df.select(columns_to_keep)

# Drop rows with missing User_id or missing review_score
df_cleaned = df_cleaned.dropna(subset=["User_id", "review_score", 'id'])

# Filter out rows with missing User_id, id, and review_score
df_cleaned = df_cleaned.filter(df_cleaned["User_id"].isNotNull() & \
                                df_cleaned["id"].isNotNull() & \
                                df_cleaned["review_score"].isNotNull() & \
                                ~isnan(df_cleaned["review_score"]))

# Convert User_id to a numeric type using a hash function
df_cleaned = df_cleaned.withColumn("User_id", (abs(hash(col("User_id"))) % (10 ** 8)).cast("integer"))

# Generate unique numeric IDs for 'id' 
df_cleaned = df_cleaned.withColumn("id_numeric", (abs(hash(col("id"))) % (10 ** 8)).cast("integer"))

# Convert review_score to a numeric type
df_cleaned = df_cleaned.withColumn("review_score", df_cleaned["review_score"].cast("float"))

# Show the first few rows of the cleaned DataFrame
df_cleaned.show()


+----------+--------+------------+----------+
|        id| User_id|review_score|id_numeric|
+----------+--------+------------+----------+
|1882931173|29152996|         4.0|   4220319|
|0595344550|99172065|         5.0|   5483819|
|0595344550|34247722|         1.0|   5483819|
|0595344550|38937258|         1.0|   5483819|
|0802841899|94390123|         4.0|  83389235|
|0802841899|27905717|         4.0|  83389235|
|B0007FIF28|98209711|         3.0|  14348244|
|B000JINSBG|93185643|         5.0|   6320134|
|0918973031|89942169|         5.0|  60162561|
|0918973031|77797397|         5.0|  60162561|
|0974289108|14092527|         3.0|  50768198|
|0974289108|66374977|         5.0|  50768198|
|B000NKGYMK|45233430|         3.0|  53508142|
|B000NKGYMK|90510181|         5.0|  53508142|
|B000NKGYMK|61065183|         3.0|  53508142|
|0789480662|49993764|         5.0|   3673254|
|0789480662|11671935|         4.0|   3673254|
|0789480662|22442162|         4.0|   3673254|
|B0000CJHIO|60885071|         4.0|

In [10]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, abs, hash, isnan, when
from pyspark.ml.regression import RandomForestRegressor, GBTRegressor
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline


# Define features assembler
vector_assembler = VectorAssembler(inputCols=["User_id", "id_numeric"], outputCol="features")

# List of models to experiment with
models = [
    ALS(userCol="User_id",
        itemCol="id_numeric",
        ratingCol="review_score",
        coldStartStrategy="drop",
        maxIter=5,
        regParam=0.01),
    RandomForestRegressor(featuresCol="features",
                           labelCol="review_score",
                           maxDepth=5,
                           numTrees=20),
    GBTRegressor(featuresCol="features",
                 labelCol="review_score",
                 maxDepth=5,
                 maxIter=20),
]

best_rmse = float("inf")
best_model = None

for model in models:
    # Create a Pipeline
    pipeline = Pipeline(stages=[vector_assembler, model])
    
    # Fit the model on the training data
    fitted_model = pipeline.fit(training)
    
    # Generate predictions on the test data
    predictions = fitted_model.transform(test)
    
    # Evaluate the model
    evaluator = RegressionEvaluator(metricName="rmse",
                                    labelCol="review_score",
                                    predictionCol="prediction")
    rmse = evaluator.evaluate(predictions)
    print(f"Root Mean Squared Error (RMSE) for {model.__class__.__name__}: {rmse}")
    
    # Update best model if current RMSE is lower
    if rmse < best_rmse:
        best_rmse = rmse
        best_model = fitted_model


Root Mean Squared Error (RMSE) for ALS: 2.890083111917061
Root Mean Squared Error (RMSE) for RandomForestRegressor: 1.139692611977118
Root Mean Squared Error (RMSE) for GBTRegressor: 1.1406907276462774
